# Instagram Multi-User Scraper
Downloads all media from Instagram posts with stats tracking, login support, and duplicate detection

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
import time
import requests
from pathlib import Path
import json
import hashlib
from datetime import datetime

In [ ]:
# Configuration
USERNAME = ""  # Your Instagram username
PASSWORD = ""  # Your Instagram password
DOWNLOAD_DIR = Path("instagram_downloads")
DOWNLOAD_DIR.mkdir(exist_ok=True)

# List of usernames to scrape
USERS_TO_SCRAPE = []  # e.g., ["user1", "user2", "user3"]

# Global tracking
downloaded_hashes = set()
stats_log = []

In [ ]:
# Setup Chrome with mobile emulation and DevTools
options = webdriver.ChromeOptions()
options.add_argument('--auto-open-devtools-for-tabs')
options.add_experimental_option('mobileEmulation', {
    'deviceName': 'iPhone 12 Pro'
})

# Selenium 4.x compatible logging setup
options.set_capability('goog:loggingPrefs', {'performance': 'ALL', 'browser': 'ALL'})

driver = webdriver.Chrome(options=options)
print("✓ Browser opened with DevTools enabled")

In [ ]:
# Login to Instagram
def login_instagram(username, password):
    driver.get('https://www.instagram.com/')
    time.sleep(3)
    
    try:
        # Wait for and click login button if on homepage
        try:
            login_link = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/accounts/login')]"))
            )
            login_link.click()
            time.sleep(2)
        except:
            pass
        
        # Enter username
        username_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "username"))
        )
        username_input.send_keys(username)
        
        # Enter password
        password_input = driver.find_element(By.NAME, "password")
        password_input.send_keys(password)
        password_input.send_keys(Keys.RETURN)
        
        time.sleep(5)
        
        # Handle "Save Your Login Info" popup
        try:
            not_now = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not now') or contains(text(), 'Not Now')]"))
            )
            not_now.click()
            time.sleep(2)
        except:
            pass
        
        # Handle "Turn on Notifications" popup
        try:
            not_now = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not Now')]"))
            )
            not_now.click()
            time.sleep(2)
        except:
            pass
        
        print("✓ Logged in successfully")
        return True
        
    except Exception as e:
        print(f"Login failed: {e}")
        return False

# Perform login
if USERNAME and PASSWORD:
    login_instagram(USERNAME, PASSWORD)
else:
    print("⚠ No login credentials provided - will try without login")

In [ ]:
# Extract post stats
def extract_post_stats():
    likes = "0"
    comments = "0"
    is_paid = False
    
    try:
        # Get likes - multiple possible selectors
        try:
            like_spans = driver.find_elements(By.CSS_SELECTOR, 'span.x1ypdohk.x1s688f.x2fvf9.xe9ewy2[role="button"]')
            if like_spans:
                likes = like_spans[0].text
        except:
            pass
        
        # Alternative: look for "likes" text
        if likes == "0":
            try:
                like_elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'like')]/span")
                for elem in like_elements:
                    text = elem.text
                    if text.replace(',', '').isdigit():
                        likes = text
                        break
            except:
                pass
    except:
        pass
    
    try:
        # Get comments
        comment_spans = driver.find_elements(
            By.CSS_SELECTOR,
            'span.xdj266r.x14z9mp.xat24cr.x1lziwak.xexx8yu.xyri2b.x18d9i69.x1c1uobl.x1hl2dhg.x16tdsg8.x1vvkbs'
        )
        for span in comment_spans:
            text = span.text
            if text.replace(',', '').isdigit():
                comments = text
                break
    except:
        pass
    
    try:
        # Check for paid partnership
        if "Paid partnership with " in driver.page_source:
            is_paid = True
    except:
        pass
    
    return likes, comments, is_paid

In [ ]:
# Click through carousel
def click_through_carousel():
    click_count = 0
    while True:
        try:
            # Try multiple selectors for next button
            next_btn = None
            selectors = [
                'button[aria-label="Next"]',
                'button[aria-label="next"]',
                'button._afxw._al46._al47',
                'button.coreSpriteRightChevron'
            ]
            
            for selector in selectors:
                try:
                    next_btn = WebDriverWait(driver, 1).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                    )
                    break
                except:
                    continue
            
            if next_btn:
                next_btn.click()
                click_count += 1
                time.sleep(0.5)
            else:
                break
                
        except (TimeoutException, NoSuchElementException):
            break
    
    return click_count

In [ ]:
# Extract media URLs from network logs (Selenium 4.x compatible)
def extract_media_urls():
    media_urls = set()
    
    try:
        # Selenium 4.x method to get logs
        logs = driver.get_log('performance')
        
        for log in logs:
            try:
                message = json.loads(log['message'])['message']
                if message['method'] == 'Network.responseReceived':
                    response = message['params']['response']
                    url = response['url']
                    mime_type = response.get('mimeType', '')
                    
                    if 'image/' in mime_type or 'video/' in mime_type:
                        if 'cdninstagram.com' in url or 'fbcdn.net' in url:
                            media_urls.add(url)
            except:
                continue
    except Exception as e:
        print(f"  Warning: Could not extract from network logs: {e}")
        print("  Trying alternative method...")
        
        # Fallback: extract from page source
        try:
            # Find img and video tags
            images = driver.find_elements(By.TAG_NAME, 'img')
            videos = driver.find_elements(By.TAG_NAME, 'video')
            
            for img in images:
                src = img.get_attribute('src')
                if src and ('cdninstagram.com' in src or 'fbcdn.net' in src):
                    media_urls.add(src)
            
            for video in videos:
                src = video.get_attribute('src')
                if src and ('cdninstagram.com' in src or 'fbcdn.net' in src):
                    media_urls.add(src)
        except Exception as e2:
            print(f"  Fallback also failed: {e2}")
    
    return media_urls

In [ ]:
# Download media with duplicate detection
def download_media(media_urls, post_dir):
    success_count = 0
    
    for idx, url in enumerate(media_urls, 1):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            
            # Check for duplicates by content hash
            content_hash = hashlib.md5(response.content).hexdigest()
            if content_hash in downloaded_hashes:
                print(f"  Skipped {idx}/{len(media_urls)}: Duplicate")
                continue
            
            downloaded_hashes.add(content_hash)
            
            # Determine file extension
            content_type = response.headers.get('content-type', '')
            if 'video' in content_type:
                ext = '.mp4'
            elif 'image' in content_type:
                ext = '.jpg'
            else:
                ext = '.jpg'  # default
            
            success_count += 1
            filename = post_dir / f"item_{success_count}{ext}"
            
            filename.write_bytes(response.content)
            print(f"  Downloaded {success_count}: {filename.name}")
            
        except Exception as e:
            print(f"  Skipped {idx}/{len(media_urls)}: {str(e)[:40]}")
    
    return success_count

In [ ]:
# Scrape a single post
def scrape_post(post_url):
    print(f"\n{'='*60}")
    print(f"Scraping: {post_url}")
    print('='*60)
    
    driver.get(post_url)
    time.sleep(3)
    driver.refresh()
    time.sleep(2)
    
    # Extract stats
    likes, comments, is_paid = extract_post_stats()
    print(f"Stats: Likes={likes}, Comments={comments}, Paid={is_paid}")
    
    # Click through carousel
    click_count = click_through_carousel()
    if click_count > 0:
        print(f"Clicked next {click_count} times")
    
    # Extract media URLs
    media_urls = extract_media_urls()
    print(f"Found {len(media_urls)} potential media items")
    
    # Create directory for this post
    post_id = post_url.rstrip('/').split('/')[-1]
    post_dir = DOWNLOAD_DIR / post_id
    post_dir.mkdir(exist_ok=True)
    
    # Download media
    success_count = download_media(media_urls, post_dir)
    
    # Log stats
    stats_log.append({
        'timestamp': datetime.now().isoformat(),
        'post_url': post_url,
        'post_id': post_id,
        'likes': likes,
        'comments': comments,
        'paid_partnership': is_paid,
        'media_downloaded': success_count
    })
    
    print(f"✓ Downloaded {success_count} unique items to '{post_dir}'")
    return success_count

In [ ]:
# Scrape user profile
def scrape_user(username, max_posts=None):
    print(f"\n{'='*60}")
    print(f"Scraping user: {username}")
    print('='*60)
    
    profile_url = f"https://www.instagram.com/{username}/"
    driver.get(profile_url)
    time.sleep(3)
    
    # Scroll to load posts
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    # Find all post links
    try:
        post_links = []
        
        # Find all links with /p/ (posts)
        links = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
        
        for link in links:
            href = link.get_attribute('href')
            if href and '/p/' in href:
                post_links.append(href)
        
        # Remove duplicates
        post_links = list(dict.fromkeys(post_links))
        
        if max_posts:
            post_links = post_links[:max_posts]
        
        print(f"Found {len(post_links)} posts")
        
        # Scrape each post
        total_downloaded = 0
        for i, post_url in enumerate(post_links, 1):
            print(f"\nPost {i}/{len(post_links)}")
            count = scrape_post(post_url)
            total_downloaded += count
            time.sleep(2)
        
        print(f"\n✓ User '{username}' complete: {total_downloaded} total items downloaded")
        return total_downloaded
        
    except Exception as e:
        print(f"Error scraping user: {e}")
        return 0

In [ ]:
# Main scraping loop - Scrape all users in the list
if USERS_TO_SCRAPE:
    for username in USERS_TO_SCRAPE:
        scrape_user(username, max_posts=None)  # Set max_posts if needed
        time.sleep(3)
else:
    print("⚠ No users to scrape. Add usernames to USERS_TO_SCRAPE list.")

In [ ]:
# Save stats log
if stats_log:
    stats_file = DOWNLOAD_DIR / "scrape_stats.json"
    with open(stats_file, 'w') as f:
        json.dump(stats_log, f, indent=2)
    print(f"\n✓ Stats saved to {stats_file}")
    print(f"\nTotal stats:")
    print(f"  Posts scraped: {len(stats_log)}")
    print(f"  Items downloaded: {sum(s['media_downloaded'] for s in stats_log)}")
    print(f"  Paid partnerships: {sum(1 for s in stats_log if s['paid_partnership'])}")
else:
    print("No stats to save")

In [ ]:
# Close browser
driver.quit()
print("\n✓ Browser closed")
print("✓ All done!")